In [ ]:
from random import randint

class Contestant:
    def __init__(self, talent_idx):
        self.talent = talent_idx
        self.no_of_votes = 0

    def simulate_voting(self, no_of_voters = 10000):
        self.restart_votes()
        
        for _ in range(no_of_voters):
            if randint() < self.talent:
                self.no_of_votes += 1

    def restart_votes(self):
        self.no_of_votes = 0

        


In [ ]:
"""Set up season"""

# Create contestants with different talent levels
def create_contestants(no_of_contestants=30):
    contestants = []
    for i in range(no_of_contestants):
        talent_idx = randint()
        contestants.append(Contestant(talent_idx))
    return contestants

def simulate_season(contestants=30, no_of_voters=10000, episodes=10, elimination_episodes=[3, 6, 10], safe_zone=[16, 10, 5]):
    for episode in range(episodes):
        print(f"Episode {episode + 1}")
        
        # Simulate voting for each contestant
        for contestant in contestants:
            contestant.simulate_voting(no_of_voters)
            print(f"Contestant {contestant.talent} received {contestant.no_of_votes} votes")
        
        # Eliminate contestants if it's an elimination episode
        if episode + 1 in elimination_episodes:
            contestants.sort(key=lambda x: x.no_of_votes, reverse=True)
            
            # Eliminate all contestants who are not on or above the safe zone
    
    print("Season simulation complete.")
